In [21]:
import pandas as pd
import numpy as np

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [9]:
##1

file ="data/data_int.txt"
with open(file,'w') as outfile:
    string = ""
    for x in range(10):
        string = string + str(x) + " "
    outfile.write(string)
with open(file, 'r') as outfile:
    print(outfile.read())

0 1 2 3 4 5 6 7 8 9 


In [16]:
##2

file = "data/data_float.txt"
mtrx = str([[0.0 for a in range(5)] for b in range(5)])
with open(file, 'w') as outfile:
    outfile.write(mtrx)
    
with open(file, mode = 'r') as f:
     print(f.read()) 

[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [36]:
##3

import csv 

file1 = "data/data_int.txt"
file2 = "data/data_float.txt"

with open(file1,'r') as in_file:
    with open('data/data_int.csv','w') as out_file:
        writer = csv.writer(out_file)
        data = in_file.read()
        data = data.replace("[","").replace("]","")
        mylist = list(int(i) for i in data.split(','))
        print(mylist)
        writer.writerow(mylist)
                                               

ValueError: invalid literal for int() with base 10: ''

In [18]:
with open('data/data_int.csv', 'r') as csv_file:
    print("csv file containing the list of int: ", csv_file.read())

csv file containing the list of int:  


2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [32]:
import json, csv

data = json.load(open('data/user_data(1).json'))
filtered = [dict for dict in data if dict['CreditCardType']=='American Express']
print("Filtered data: \n", filtered)

writer = csv.DictWriter(open('data/jsons.csv', 'w'), filtered[0].keys())
for dict in filtered:
    writer.writerow(dict)

Filtered data: 
 [{'ID': '2', 'JobTitle': 'Investment  Advisor', 'EmailAddress': 'Clint_Thorpe5003@bulaffy.com', 'FirstNameLastName': 'Clint Thorpe', 'CreditCard': '7083-8766-0251-2345', 'CreditCardType': 'American Express'}, {'ID': '12', 'JobTitle': 'Retail Trainee', 'EmailAddress': 'Phillip_Carpenter9505@famism.biz', 'FirstNameLastName': 'Phillip Carpenter', 'CreditCard': '3657-0088-0820-5247', 'CreditCardType': 'American Express'}, {'ID': '28', 'JobTitle': 'Project Manager', 'EmailAddress': 'Russel_Graves1378@extex.org', 'FirstNameLastName': 'Russel Graves', 'CreditCard': '6718-4818-8011-6024', 'CreditCardType': 'American Express'}, {'ID': '39', 'JobTitle': 'Stockbroker', 'EmailAddress': 'Leanne_Newton1268@typill.biz', 'FirstNameLastName': 'Leanne Newton', 'CreditCard': '5438-0816-4166-4847', 'CreditCardType': 'American Express'}, {'ID': '57', 'JobTitle': 'Budget Analyst', 'EmailAddress': 'Tony_Giles1960@iatim.tech', 'FirstNameLastName': 'Tony Giles', 'CreditCard': '8130-3425-7573-7

3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [23]:
import pandas as pd

df = pd.read_csv("data/mushrooms_categorized.csv")

means = df.groupby(["class"]).agg('mean')

means

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
class,,,,,,,,,,,,,,,,,,,,,
0,3.266160,1.615970,4.581749,0.653992,4.334601,0.954373,0.285171,0.068441,6.622624,0.615970,...,1.798479,6.098859,6.064639,0.0,1.931559,1.125475,3.007605,3.201521,3.283270,1.148289
1,3.436159,2.055158,4.421859,0.159346,3.940756,0.995403,0.028601,0.567926,2.863636,0.514811,...,1.394280,5.512768,5.504597,0.0,2.002043,1.009193,1.522983,4.021450,4.031665,1.895812


4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [31]:
with open('data/credit_card.dat','rb') as file:
    while True:
        file_content=file.readline()
        
        ints=[]
        try:
            for i in range(1,20*7,7):
                if (i==34 or i==64 or i==94):
                    continue
                ints.append(int(file_content[i:i+6], 2))
        except ValueError:
            break
        print("-". join(str(j) for j in ints))

5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3

In [29]:
import csv 
import pandas as pd
import matplotlib.pyplot as plt

